# Concurrency: shared memory; POSIX threads
_COSC 208, Introduction to Computer Systems, Spring 2025_

## Threads and memory

* Threads are multiple execution contexts within the **same process**
    * Processes are multiple execution contexts within the **same machine**
* Because threads are within the same process, they share all of the process’s resources — memory and CPU time
* Consequently, two threads can update the same variable

_Example_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *thread1_main(void *arg) {
/* 4*/      int *x = (int *)arg;
/* 5*/      *x += 1;
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  void *thread2_main(void *arg) {
/* 9*/      int *y = (int *)arg;
/*10*/      *y += 2;
/*11*/      return NULL;
/*12*/  }
/*13*/  int main() {
/*14*/      int *z = malloc(sizeof(int));
/*15*/      *z = 0;
/*16*/      // Create thread running thread1_main(z)
/*17*/      // Create thread running thread2_main(z)
/*18*/      // Wait for threads to finish
/*19*/      printf("z is %d\n", *z);
/*20*/  }

```
z is 3
```

<p style="height:25em;"></p>

Q1: _What are all possible outputs produced by this program, assuming the threads don't overlap each other's operations?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  void *increment(void *arg) {
/* 4*/      int *num = (int*)arg;
/* 5*/      *num += 1;
/* 6*/      return NULL;
/* 7*/  }
/* 8*/  void *zero(void *arg) {
/* 9*/      int *num = (int*)arg;
/*10*/      *num = 0;
/*11*/      return NULL;
/*12*/  }
/*13*/  int main() {
/*14*/      int *i = malloc(sizeof(int));
/*15*/      *i = 5;
/*16*/      // Create thread running increment(i)
/*17*/      // Create thread running zero(i)
/*18*/      // Wait for threads to finish
/*19*/      printf("i=%d\n", *i);
/*20*/  }

```
i=0
```
OR
```
i=1
```

<p style="height:8em;"></p>

## Pthreads API

* Can create and wait for threads to finish, just like processes, but API is different
* Use the pthreads library—`#include <pthread.h>`
* `int pthread_create(pthread_t *thread, const pthread_attr_t *attr, void *(*start_routine)(void*), void * arg)`
    * `thread`—a struct that stores metadata for the thread
    * `attr`—configuration settings for the thread
    * `start_routine`—the function to start executing when the thread starts
        * Pass a pointer to a function
    * `arg`—an argument passed to the aforementioned function
* `int pthread_join(pthread_t thread, void **value_ptr)`
    * `thread`—the same struct passed at thread creation; used to identify the thread we want to wait for
    * `value_ptr`—the location where the function return value should be stored
        * Notice it’s a pointer to a void pointer and the `start_routine` function specified in `pthread_create` returns a void pointer

<p style="height:22em;"></p>

Q3: _What are all possible outputs produced by this program?_

In [1]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <pthread.h>
/* 4*/  void *printer(void *arg) {
/* 5*/      char *ch = (char*)arg;
/* 6*/      printf("I am %c\n", *ch);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  int main() {
/*10*/      pthread_t thread1, thread2;
/*11*/      char *ch = malloc(sizeof(char));
/*12*/      *ch = 'P';
/*13*/      pthread_create(&thread1, NULL, &printer, ch);
/*14*/      pthread_join(thread1, NULL);
/*15*/      *ch = 'Q';
/*16*/      pthread_create(&thread2, NULL, &printer, ch);
/*17*/      pthread_join(thread2, NULL);
/*18*/  }

I am P
I am Q


<p style="height:6em;"></p>

Q4: _What are all possible outputs produced by this program?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <pthread.h>
/* 4*/  void *printer(void *arg) {
/* 5*/      char *ch = (char*)arg;
/* 6*/      printf("I am %c\n", *ch);
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  int main() {
/*10*/      pthread_t thread1, thread2;
/*11*/      char *ch = malloc(sizeof(char));
/*12*/      *ch = 'M';
/*13*/      pthread_create(&thread1, NULL, &printer, ch);
/*14*/      *ch = 'N';
/*15*/      pthread_create(&thread2, NULL, &printer, ch);
/*16*/      pthread_join(thread1, NULL);
/*17*/      pthread_join(thread2, NULL);
/*18*/  }

```
I am M
I am N
```
OR
```
I am N
I am N
```

<div style="height:6em;"></div>

Q5: _Consider the following program:_

In [ ]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <string.h>
/* 4*/  void word_count(char *str) {
/* 5*/      int count = 1;
/* 6*/      for (int i = 0; i < strlen(str); i++) {
/* 7*/          if (str[i] == ' ') {
/* 8*/              count++;
/* 9*/          }
/*10*/     }
/*11*/     printf("%d words\n", count);
/*12*/  }
/*13*/  int main(int argc, char *argv[]) {
/*14*/      char *str = "I love CS";
/*15*/      pthread_t thr;
/*16*/      pthread_create(thr, NULL, &word_count, str);
/*17*/      pthread_join(thr);
/*18*/  }

_Compiling this program with `gcc` results in the following errors:_
```
buggy_noreturn.c:16:20: warning: passing argument 1 of ‘pthread_create’ 
makes pointer from integer without a cast [-Wint-conversion]
   16 |     pthread_create(thr, NULL, &word_count, str);
      |                    ^~~
      |                    pthread_t {aka long unsigned int}
/usr/include/pthread.h:202:50: note: expected ‘pthread_t * restrict’ 
{aka ‘long unsigned int * restrict’} but argument is of type ‘pthread_t’ 
{aka ‘long unsigned int’}
  202 | extern int pthread_create (pthread_t *__restrict __newthread,
      |                            ~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~
buggy_noreturn.c:16:31: warning: passing argument 3 of ‘pthread_create’ 
from incompatible pointer type [-Wincompatible-pointer-types]
   16 |     pthread_create(thr, NULL, &word_count, str);
      |                               ^~~~~~~~~~~
      |                               void (*)(char *)
/usr/include/pthread.h:204:36: note: expected ‘void * (*)(void *)’ but 
argument is of type ‘void (*)(char *)’
  204 |                            void *(*__start_routine) (void *),
      |                            ~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~
buggy_noreturn.c:17:5: error: too few arguments to function ‘pthread_join’
   17 |     pthread_join(thr);
      |     ^~~~~~~~~~~~
/usr/include/pthread.h:219:12: note: declared here
  219 | extern int pthread_join (pthread_t __th, void **__thread_return);
      |            ^~~~~~~~~~~~
```

_How would you change the code to fix these problems?_

* Need to pass `&thr` to `pthread_create` (instead of `thr`) on line 16
* Function executed by a thread must return `void *` and take a single `void *` parameter; replace lines 6-7 with:
    ```C
    void *word_count(void *arg) {
        char *str = (char *)arg;
    ```
    Also add after line 11:
    ```C
    return NULL;
    ```
* Add an additional parameter to `pthread_join` one line 17:
    ```C
    pthread_join(thr, NULL);
    ``` 

<div style="height:1em;"></div>

A program contains the following functions:
```C
void *dbl(void *arg) {
    int *t = (int *)arg;
    *t = *t * 2;
}

void *inc(void *arg) {
    int *t = (int *)arg;
    *t = *t + 1;
}
```
_For each of the following main methods, list **all possible outputs** the program could produce. Assume threads are only preempted if they become blocked waiting for other threads._

In [ ]:
// Q7
int main() {
    int *total = malloc(sizeof(int));
    *total = 3;
    pthread_t thrA, thrB;
    pthread_create(&thrA, NULL, &dbl, total);
    pthread_create(&thrB, NULL, &inc, total);
    pthread_join(thrA, NULL);
    pthread_join(thrB, NULL);
    printf("%d\n", total);
}

* `7` (if `thrB` runs after `thrA` finishes)
* `8` (if `thrA` runs after `thrB` finishes)

<p style="height:10em;"></p>

In [ ]:
// Q8
int main() {
    int *total = malloc(sizeof(int));
    *total = 3
    pthread_t thrA, thrB;
    pthread_create(&thrA, NULL, &inc, total);
    pthread_join(thrA, NULL);
    pthread_create(&thrB, NULL, &dbl, total);
    pthread_join(thrB, NULL);
    printf("%d\n", total);
}

* `8` (`thrA` is joined, i.e., must finish, before `thrB` is created) 

<p style="height:10em;"></p>

Q9: _Consider the following program:_

In [ ]:
/* 1*/  #include <ctype.h>
/* 2*/  #include <pthread.h>
/* 3*/  #include <stdio.h>
/* 4*/  #include <stdlib.h>
/* 5*/  #include <string.h>
/* 6*/  int count_upper(char *str) {
/* 7*/      int count = 0;
/* 8*/      for (int i = 0; i < strlen(str); i++) {
/* 9*/          if (isupper(str[i])) {
/*10*/              count++;
/*11*/          }
/*12*/      }
/*13*/      return count;
/*14*/  }
/*15*/  int main(int argc, char *argv[]) {
/*16*/      if (argc < 2) {
/*17*/          printf("Error: provide a string\n");
/*18*/          return 1;
/*19*/      }
/*20*/      char *str = argv[1];
/*21*/      pthread_t thr;
/*22*/      pthread_create(thr, NULL, &count_upper, str);
/*23*/      int count = 0;
/*24*/      pthread_join(thr, &count);
/*25*/      printf("There are %d uppercase letters\n", count);
/*26*/  }

_Compiling this program results in the following warnings:_
```
buggy.c:22:20: warning: incompatible integer to pointer conversion passing 
'pthread_t' (aka 'unsigned long') to parameter of type 'pthread_t *' (aka 
'unsigned long *'); take the address with & [-Wint-conversion]
    pthread_create(thr, NULL, &count_upper, str);
                   ^~~
                   &
/usr/include/pthread.h:198:50: note: passing argument to parameter 
'__newthread' here
extern int pthread_create (pthread_t *__restrict __newthread,
                                                 ^

buggy.c:22:31: warning: incompatible function pointer types passing 
'int (*)(char *)' to parameter of type 'void *(*)(void *)' 
[-Wincompatible-function-pointer-types]
    pthread_create(thr, NULL, &count_upper, str);
                              ^~~~~~~~~~~~
/usr/include/pthread.h:200:15: note: passing argument to parameter 
'__start_routine' here
                           void *(*__start_routine) (void *),
                                   ^

buggy.c:24:23: warning: incompatible pointer types passing 
'int *' to parameter of type 'void **' [-Wincompatible-pointer-types]
    pthread_join(thr, &count);
                      ^~~~~~
/usr/include/pthread.h:215:49: note: passing argument to parameter 
'__thread_return' here
extern int pthread_join (pthread_t __th, void **__thread_return);
                                                ^
3 warnings generated.
```
_How would you change the code to fix these problems?_

* Need to pass `&thr` to `pthread_create` (instead of `thr`) on line 22
* Function executed by a thread must return `void *` and take a single `void *` parameter; replace lines 6-7 with:
    ```C
    void *count_upper(void *arg) {
        char *str = (char *)arg;
        int *count = malloc(sizeof(int));
        *count = 0;
    ```
    Also replace line 10 with:
    ```C
    *count++;
    ```
* Need to pass a double pointer to `pthread_join` on line 24; replace lines 23-25 with:
    ```C
    int *count = NULL;
    pthread_join(thr, &count);
    printf("There are %d uppercase letters\n", *count);
    ``` 